In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score

from xgboost import XGBClassifier
import xgboost as xgb
from sklearn.utils.class_weight import compute_sample_weight, compute_class_weight

import optuna
import joblib  # to save best model

from tqdm import tqdm
tqdm.pandas(desc="Progress")

from sklearn.pipeline import Pipeline
import mlflow
import mlflow.sklearn

from sklearn.metrics import (
    f1_score, accuracy_score, precision_score,
    recall_score, confusion_matrix, classification_report
)



d:\code\interview_assignment\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def plot_confusion_matrix(y_test, y_pred, target_column):
    # Compute confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    
    # Display class labels if you know them
    labels = df[target_column].astype('category').cat.categories.tolist()
    
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels)
    plt.title("Confusion Matrix")
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.show()

def plot_feature_importances(model, feature_names):
    # Get feature importances
    importances = model.feature_importances_
    
    # Plot feature importances
    plt.figure(figsize=(10, 6))
    sns.barplot(x=importances, y=feature_names, palette="viridis")
    plt.title("XGBoost Feature Importances")
    plt.xlabel("Importance Score")
    plt.ylabel("Features")
    plt.tight_layout()
    plt.show()

def calculate_f1_score(y_true, y_pred):
    from sklearn.metrics import f1_score
    # Macro F1 Score: treats all classes equally
    f1_macro = f1_score(y_test, y_pred, average='macro')

    # Micro F1 Score: globally by counting total true positives/negatives
    f1_micro = f1_score(y_test, y_pred, average='micro')

    # Weighted F1 Score: accounts for support (number of true instances per class)
    f1_weighted = f1_score(y_test, y_pred, average='weighted')

    print(f"F1 Score (Macro): {f1_macro:.4f}")
    print(f"F1 Score (Micro): {f1_micro:.4f}")
    print(f"F1 Score (Weighted): {f1_weighted:.4f}")


def f1_per_class(y_test, y_pred, class_labels = None):
    # Compute F1 score per class
    class_f1_scores = f1_score(y_test, y_pred, average=None)

    # Set default class labels if not provided
    if class_labels is None:
        class_labels = sorted(set(y_test))

    # Plot
    plt.figure(figsize=(8, 5))
    sns.barplot(x=class_labels, y=class_f1_scores, palette='rocket')
    plt.title("F1 Score per Class")
    plt.xlabel("Class")
    plt.ylabel("F1 Score")
    plt.ylim(0, 1)
    plt.tight_layout()
    plt.show()

In [3]:
df = pd.read_csv('./data/final_data.csv')

In [4]:
df.head()

,high_activity,packets_ratio,source_port,low_activity,bytes_ratio,destination_port,nat_destination_port,elapsed_time_(sec),nat_source_port,burst_transfer,bytes_per_sec,action
0,0,0.500000,57222,0,1.119048,53,53,30,54587,0,5.900000,allow
1,0,1.000000,56258,0,0.504891,3389,3389,17,56258,0,280.470572,allow
2,0,0.500000,6881,0,0.975207,50321,50321,1199,43265,0,0.198499,allow
3,0,1.000000,50553,0,0.760847,3389,3389,17,50553,0,195.705871,allow
4,0,0.684211,50002,0,0.364781,443,443,16,45848,0,1584.874901,allow


In [5]:
df.rename(columns={'action': 'target'}, inplace=True)

In [6]:
# Encode the target variable
df['target'] = df['target'].astype('category').cat.codes

In [7]:
# separating data in to features and target variable
X = df.drop(columns=['target'])
y = df['target']

print(f"Shape of X: {X.shape}, Shape of y: {y.shape}")

Shape of X: (57170, 11), Shape of y: (57170,)


In [8]:
# splitting data into train and test sets and validating the split

# Initial split: 80% train_val, 20% test
X_train_val, X_test, y_train_val, y_test = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,
    random_state=42
)

# Split train_val into 75% train and 25% val → overall: 60% train, 20% val, 20% test
X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val,
    test_size=0.25,  # 0.25 * 0.8 = 0.2 of total
    stratify=y_train_val,
    random_state=42
)
print(f"Shape of X_train: {X_train.shape}, Shape of y_train: {y_train.shape}")
print(f"Shape of X_val: {X_val.shape}, Shape of y_val: {y_val.shape}")
print(f"Shape of X_test: {X_test.shape}, Shape of y_test: {y_test.shape}")

Shape of X_train: (34302, 11), Shape of y_train: (34302,)
Shape of X_val: (11434, 11), Shape of y_val: (11434,)
Shape of X_test: (11434, 11), Shape of y_test: (11434,)


In [9]:
y_train.value_counts(), y_val.value_counts(), y_test.value_counts()

(target
 0    22463
 2     6981
 1     4826
 3       32
 Name: count, dtype: int64,
 target
 0    7488
 2    2327
 1    1608
 3      11
 Name: count, dtype: int64,
 target
 0    7488
 2    2327
 1    1608
 3      11
 Name: count, dtype: int64)

In [26]:

def objective(trial):
    # 📦 Hyperparameters
    params = {
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "n_estimators": trial.suggest_int("n_estimators", 100, 500),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "gamma": trial.suggest_float("gamma", 0, 5),
        "reg_lambda": trial.suggest_float("reg_lambda", 0.1, 10),
        "reg_alpha": trial.suggest_float("reg_alpha", 0.1, 10),
        "objective": "multi:softprob",
        "num_class": len(np.unique(y_train)),
        "eval_metric": "mlogloss",
        "random_state": 42,
    }
    
    # Model definition
    model = xgb.XGBClassifier(**params) # Direct XGBClassifier, not in Pipeline for easier callback handling

    # Compute sample weights dynamically for each trial
    sample_weights = compute_sample_weight(class_weight="balanced", y=y_train)
    
    model.fit(X_train, y_train,
              sample_weight=sample_weights,
              eval_set=[(X_val, y_val)], # Use X_test, y_test as validation set for early stopping
            #   early_stopping_rounds=50,    # Stop if validation metric doesn't improve for 50 rounds
              verbose=False                # Suppress verbose output during tuning
             )

    # Predict
    y_pred = model.predict(X_test)
    f1_weighted = f1_score(y_test, y_pred, average='macro')

    return f1_weighted


In [11]:
# Optuna Optimization 
print("\nStarting Optuna hyperparameter tuning...")
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=42))
study.optimize(objective, n_trials=50, show_progress_bar=True) # Reduced trials for demonstration

[I 2025-07-26 12:23:38,012] A new study created in memory with name: no-name-bf6e694c-1639-4739-909b-91c982f9bdbd



Starting Optuna hyperparameter tuning...


Best trial: 0. Best value: 0.92501:   2%|▏         | 1/50 [00:06<05:37,  6.88s/it]

[I 2025-07-26 12:23:44,888] Trial 0 finished with value: 0.9250103078931113 and parameters: {'learning_rate': 0.11861663446573512, 'max_depth': 10, 'n_estimators': 393, 'subsample': 0.7993292420985183, 'colsample_bytree': 0.5780093202212182, 'gamma': 0.7799726016810132, 'reg_lambda': 0.6750277604651747, 'reg_alpha': 8.675143843171858}. Best is trial 0 with value: 0.9250103078931113.


Best trial: 1. Best value: 0.932844:   4%|▍         | 2/50 [00:08<03:11,  3.99s/it]

...

[I 2025-07-26 12:28:40,962] Trial 47 finished with value: 0.9486165123981306 and parameters: {'learning_rate': 0.08901379915169082, 'max_depth': 7, 'n_estimators': 449, 'subsample': 0.9468333362489876, 'colsample_bytree': 0.9031483836902796, 'gamma': 1.5485601338104753, 'reg_lambda': 4.983849338092982, 'reg_alpha': 4.943670816615743}. Best is trial 47 with value: 0.9486165123981306.


Best trial: 47. Best value: 0.948617:  98%|█████████▊| 49/50 [05:10<00:07,  7.77s/it]

[I 2025-07-26 12:28:48,679] Trial 48 finished with value: 0.9234608945954541 and parameters: {'learning_rate': 0.08872527917837253, 'max_depth': 7, 'n_estimators': 396, 'subsample': 0.9128350977395951, 'colsample_bytree': 0.911202969961542, 'gamma': 1.5168665510700026, 'reg_lambda': 4.750954549675857, 'reg_alpha': 4.955261367011411}. Best is trial 47 with value: 0.9486165123981306.


Best trial: 47. Best value: 0.948617: 100%|██████████| 50/50 [05:18<00:00,  6.36s/it]

[I 2025-07-26 12:28:56,150] Trial 49 finished with value: 0.888069424624257 and parameters: {'learning_rate': 0.048024962433257996, 'max_depth': 10, 'n_estimators': 376, 'subsample': 0.7699259306689287, 'colsample_bytree': 0.9460388522724476, 'gamma': 1.9873455783493728, 'reg_lambda': 4.945645852929802, 'reg_alpha': 7.597822372591748}. Best is trial 47 with value: 0.9486165123981306.


In [14]:
print("\n--- Optuna Optimization Results ---")
print(f"Number of finished trials: {len(study.trials)}")
print(f"Best trial number: {study.best_trial.number}")
print(f"Best F1-score: {study.best_value:.4f}")
print("Best hyperparameters:")
for key, value in study.best_trial.params.items():
    print(f"  {key}: {value}")


--- Optuna Optimization Results ---
Number of finished trials: 50
Best trial number: 47
Best F1-score: 0.9486
Best hyperparameters:
  learning_rate: 0.08901379915169082
  max_depth: 7
  n_estimators: 449
  subsample: 0.9468333362489876
  colsample_bytree: 0.9031483836902796
  gamma: 1.5485601338104753
  reg_lambda: 4.983849338092982
  reg_alpha: 4.943670816615743


In [13]:
# --- Optimization Results ---
try:
    fig_history = optuna.visualization.plot_optimization_history(study)
    fig_history.update_layout(title="Optuna Optimization History (F1-Score)")
    fig_history.show()

    fig_importance = optuna.visualization.plot_param_importances(study)
    fig_importance.update_layout(title="Hyperparameter Importances")
    fig_importance.show()

    fig_slice = optuna.visualization.plot_slice(study)
    fig_slice.update_layout(title="Slice Plot of Hyperparameters")
    fig_slice.show()
except Exception as e:
    print(f"\nCould not generate Optuna visualizations. Error: {e}")
    print("This might happen if Plotly is not installed or if running in an environment without a browser/display for interactive plots.")
    print("You can install plotly using: pip install plotly")



In [15]:
study.best_trial.params.items()

dict_items([('learning_rate', 0.08901379915169082), ('max_depth', 7), ('n_estimators', 449), ('subsample', 0.9468333362489876), ('colsample_bytree', 0.9031483836902796), ('gamma', 1.5485601338104753), ('reg_lambda', 4.983849338092982), ('reg_alpha', 4.943670816615743)])

## final Model

In [18]:
best_params = study.best_trial.params
print("\nBest Hyperparameters:")
for key, value in best_params.items():
    print(f"  {key}: {value}")  



Best Hyperparameters:
  learning_rate: 0.08901379915169082
  max_depth: 7
  n_estimators: 449
  subsample: 0.9468333362489876
  colsample_bytree: 0.9031483836902796
  gamma: 1.5485601338104753
  reg_lambda: 4.983849338092982
  reg_alpha: 4.943670816615743


In [19]:
# read the final data
print("\nReading final data for model training...")
df = pd.read_csv('./data/final_data.csv')

# renaming the target column to 'target' for consistency
df.rename(columns={'action': 'target'}, inplace=True)

# Encode the target variable
df['target'] = df['target'].astype('category').cat.codes

# separating data in to features and target variable
X = df.drop(columns=['target'])
y = df['target']

print(f"Shape of X: {X.shape}, Shape of y: {y.shape}")


Reading final data for model training...
Shape of X: (57170, 11), Shape of y: (57170,)


In [20]:
# Initial split: 80% train_val, 20% test
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,
    random_state=41
)

print(f"Shape of X_train: {X_train.shape}, Shape of y_train: {y_train.shape}")
print(f"Shape of X_test: {X_test.shape}, Shape of y_test: {y_test.shape}")


Shape of X_train: (45736, 11), Shape of y_train: (45736,)
Shape of X_test: (11434, 11), Shape of y_test: (11434,)


In [22]:
# Model definition
final_model = xgb.XGBClassifier(**best_params)

# Compute sample weights dynamically for each trial
sample_weights = compute_sample_weight(class_weight="balanced", y=y_train)

final_model.fit(X_train, y_train,
            sample_weight=sample_weights,
            verbose=True
            )


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.9031483836902796, device=None,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, feature_weights=None,
              gamma=1.5485601338104753, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.08901379915169082,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=7, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=449, n_jobs=None,
              num_parallel_tree=None, ...)

In [23]:
# predict
y_pred = final_model.predict(X_test)

In [42]:
def classification_report_and_confusion_matrix(y_true, y_pred, labels):
    # 📝 Classification report
    report_text = classification_report(y_true, y_pred, target_names=labels)
    ## save report to a text file
    with open("./data/classification_report.txt", "w") as f:
        f.write(report_text)
        
    # 🔄 Confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=labels, yticklabels=labels)
    plt.title("Confusion Matrix")
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.tight_layout()
    plt.savefig("./data/confusion_matrix.png")
    plt.close()
    # mlflow.log_artifact("./data/confusion_matrix.png")

    # 🎯 F1 Score per class
    f1_per_class = f1_score(y_true, y_pred, average='macro')
    plt.figure(figsize=(8, 5))
    # sns.barplot(x=labels, y=f1_per_class, palette="rocket")
    sns.barplot(x=labels, y=f1_per_class, hue=labels, palette="rocket", legend=False)
    plt.title("F1 Score per Class")
    plt.ylim(0, 1)
    plt.tight_layout()
    plt.savefig("./data/f1_per_class.png")
    plt.close()
    # mlflow.log_artifact("./data/f1_per_class.png")

    return f1_per_class



In [43]:
labels = df['target'].astype('category').cat.categories.tolist()
labels = [str(label) for label in labels]  # Convert to string for better readability

In [44]:
classification_report_and_confusion_matrix(y_test, y_pred, labels=labels)

0.8731127837523902

In [45]:
## saving the model
joblib.dump(final_model, './data/final_model.pkl')


['./data/final_model.pkl']

In [46]:
df.columns

Index(['high_activity', 'packets_ratio', 'source_port', 'low_activity',
       'bytes_ratio', 'destination_port', 'nat_destination_port',
       'elapsed_time_(sec)', 'nat_source_port', 'burst_transfer',
       'bytes_per_sec', 'target'],
      dtype='object')

In [48]:
final_model.get_booster().feature_names


['high_activity',
 'packets_ratio',
 'source_port',
 'low_activity',
 'bytes_ratio',
 'destination_port',
 'nat_destination_port',
 'elapsed_time_(sec)',
 'nat_source_port',
 'burst_transfer',
 'bytes_per_sec']